In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from datetime import *
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX


train_df=pd.read_csv('./data/train_v2.csv')
wkd_df=pd.read_csv('./data/wkd_v1.csv')
wkd_df=wkd_df.rename(columns={'ORIG_DT':'date'})
tmp=train_df[['date','post_id','amount']].groupby(['date','post_id'],sort=False).agg('sum')
train_day_df=pd.DataFrame(tmp).reset_index()
train_day_df_A=train_day_df[train_day_df['post_id']=='A'].reset_index(drop=True)
train_day_df_B=train_day_df[train_day_df['post_id']=='B'].reset_index(drop=True)
train_day_df_A=train_day_df_A.merge(wkd_df)
train_day_df_B=train_day_df_B.merge(wkd_df)
def get_frt(df):
#     df['WKD_TYP_CD']=df['WKD_TYP_CD'].map({'WN':0,'SN': 1, 'NH': 1, 'SS': 1, 'WS': 0})
    df['date'] = pd.to_datetime(df['date'])
    df['year']=df['date'].dt.year
    df['month']=df['date'].dt.month
    df['day']=df['date'].dt.day
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofweek'] = df['date'].dt.dayofweek
    df.drop(['post_id'],axis=1,inplace=True)
    return df
train_day_df_A=get_frt(train_day_df_A)
train_day_df_B=get_frt(train_day_df_B)
train_day_df_A['amount']=train_day_df_A['amount']/1e4
train_day_df_B['amount']=train_day_df_B['amount']/1e4


'''
对于任务一岗位A:用到了2018年7月到12月、2020年7月到11月清洗后的数据
对于任务一岗位B:用到了2018年10-12月、2020年10-12月清洗后的数据
'''
#岗位A
a2018 = train_day_df_A[(train_day_df_A.year==2018)&(train_day_df_A.month>6)].copy().reset_index(drop=True)
a2019 = train_day_df_A[(train_day_df_A.year==2019)&(train_day_df_A.month>6)].copy().reset_index(drop=True)
a2020 = train_day_df_A[(train_day_df_A.year==2020)&(train_day_df_A.month>6)].copy().reset_index(drop=True)
lst2018 = [0,1,2,3,4,5,6,7]+list(range(85,106))+[183]
lst2019 = [0,1,2,3,4,5,6]+list(range(84,105)) +[182,183]
lst2020 = [0,1,2,3,4]+list(range(82,103))
trainlst = [it for it in a2018.index if it not in lst2018]
a2018 = a2018.loc[trainlst].reset_index(drop=True)
trainlst = [it for it in a2019.index if it not in lst2019]
a2019 = a2019.loc[trainlst].reset_index(drop=True)
trainlst = [it for it in a2020.index if it not in lst2020]
a2020 = a2020.loc[trainlst].reset_index(drop=True)
for item in [a2018,a2019,a2020]:
    uperbound= item[(item.WKD_TYP_CD.isin(['WN','WS']))&(item.amount>=0.1)&(item.amount<4)]['amount'].mean()
    lowbound = item[(item.WKD_TYP_CD.isin(['SN','SS']))&(item.amount>=0.1)&(item.amount<4)]['amount'].mean()
    for it in item[item.WKD_TYP_CD.isin(['WN','WS'])].index:
        if item.loc[it,'amount']<0.1 or item.loc[it,'amount']>4:
            item.loc[it,'amount'] = uperbound
    for it in item[item.WKD_TYP_CD.isin(['SN','SS'])].index:
        if item.loc[it,'amount']<0.1:
            item.loc[it,'amount'] = lowbound
tmp_A = a2018.append(a2020)
tmp_A = tmp_A.sort_values(by='date').reset_index(drop=True)
# tmp_A = tmp.set_index('date')

# tmp['amount'].to_csv('data/train_task1_A_longterm_18_20.csv')
#岗位B
b2018 = train_day_df_B[(train_day_df_B.year==2018)&(train_day_df_B.month>9)].copy().reset_index(drop=True)
b2019 = train_day_df_B[(train_day_df_B.year==2019)&(train_day_df_B.month>9)].copy().reset_index(drop=True)
b2020 = train_day_df_B[(train_day_df_B.year==2020)&(train_day_df_B.month>9)].copy().reset_index(drop=True)
for item in [b2018,b2019,b2020]:
    uperbound= item[(item.dayofweek.isin([0,1,2,3,4]))&(item.amount>0.1)]['amount'].mean()
    for it in item.index:
        if item.loc[it,'dayofweek'] in [0,1,2,3,4] and item.loc[it,'amount']<=0.1:
            item.loc[it,'amount'] = uperbound
for item in [b2019]:
    uperbound= item[(item.dayofweek.isin([0,1,2,3,4]))&(item.amount>0.1)]['amount'].mean()
    for it in item.index:
        if item.loc[it,'month'] >9 and item.loc[it,'amount']>0.4:
            item.loc[it,'amount'] = uperbound
tmp_B = b2018.loc[range(14,91)].append(b2020.loc[range(11,61)])
tmp_B = tmp_B.sort_values(by='date').reset_index(drop=True)
# tmp_B = tmp.set_index('date')
# tmp['amount'].to_csv('data/train_task1_B_shortterm.csv')

In [20]:
def build_arima(ts_log,order,seasonal_order):

    model = SARIMAX(ts_log, order=order, seasonal_order=seasonal_order, trend='c')
    results_fitted = model.fit(disp=7)
    fit_values = results_fitted.fittedvalues
    print(results_fitted.summary())
    fc = results_fitted.forecast(31)
    conf = None
    return fit_values, fc, conf


def trans_back(trend,seasonal):
    trend_pre = pd.DataFrame(index=trend.index,columns=['amount'])
    trend_pre.index.name = 'date'
    begin = datetime(2020, 12, 1)
    seasonal_ = []
    for i in range(7):
        seasonal_.append(seasonal[seasonal.index.dayofweek==i].mean())
    # print(seasonal_)
    for i,item in enumerate(trend_pre.index):
        day = begin +timedelta(days=i)
        trend_pre.loc[item,'amount'] = trend.loc[item,'amount'] + seasonal_[day.weekday()]
    print(trend_pre)
    # trend_pre['amount'] = np.exp(trend_pre['amount'])
    return trend_pre
    # print(trend_pre)

def get_frt_test(df):
    df['date'] = pd.to_datetime(df['date'])
    df['year']=df['date'].dt.year
    df['month']=df['date'].dt.month
    df['day']=df['date'].dt.day
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofweek'] = df['date'].dt.dayofweek
    return df


if __name__ == '__main__':
    # A岗
    ts = pd.Series(tmp_A['amount'].values*1e1,index=pd.date_range("1/1/2018",periods=len(tmp_A['amount'].values)))
    ts_log = np.log(ts)
    ts_log[np.isnan(ts_log)] = 0
    ts_log[np.isinf(ts_log)] = 0
    order1 = (14, 1, 0)
    seasonal_order1 = (0, 1, 0, 7)
    _fittedvalues, fc, _conf= build_arima(ts_log,order1,seasonal_order1)
    fc_exp = np.exp(fc)
    future_index = pd.date_range(start=ts.index[-1], freq='D', periods=32)
    future_ARIMA = pd.Series(fc_exp, index=future_index)
    prea = pd.DataFrame(columns=['date', 'amount'])
    prea['date'] = pd.date_range("2020/12/1", freq='D', periods=31)
    prea['amount'] = future_ARIMA.values[1:]
    prea['amount']= prea['amount']*1e3
    #根据往年的规律对缩小误差
    #2018年12月工作日比2018年11月工作日平均多2289
    #2019年12月工作日比2018年11月工作日平均多1152
    #2020年11月工作日平均18215，12月平均大概在19355-20500之间
    #双休日基本没有变化
    prea = get_frt_test(prea)
    for item in prea.index:
        if prea.loc[item, 'dayofweek'] in [0, 1, 2, 3, 4]:
            prea.loc[item, 'amount'] -= 1500
        else:
            prea.loc[item, 'amount'] -= 900
    print(prea[prea.dayofweek.isin([0, 1, 2, 3, 4])]['amount'].mean())



    #B岗
    ts = pd.Series(tmp_B['amount'].values*1e1,index=pd.date_range("1/1/2018",periods=len(tmp_B['amount'].values)))
    print(ts)
    ts_log = np.log(ts)
    ts_log[np.isnan(ts_log)] = 0
    ts_log[np.isinf(ts_log)] = 0
    decomposition = seasonal_decompose(ts_log, freq=14)
    trend = decomposition.trend
    seasonal = decomposition.seasonal
    order = (1,0,1)
    seasonal_order = (1, 0, 1, 7)
    _, fc, _ = build_arima(trend,order,seasonal_order)
    future_index = pd.date_range(start=ts.index[-1], freq='D', periods=32)
    future_ARIMA = pd.Series(fc, index=future_index)
    preb_trend = pd.DataFrame(columns=['date', 'amount'])
    preb_trend['date'] = pd.date_range("2020/12/1", freq='D', periods=31)
    preb_trend['amount'] = future_ARIMA.values[1:]
    preb = trans_back(preb_trend, seasonal)
    preb = preb.reset_index()
    preb['amount'] = preb['amount'].apply(lambda x:np.exp(x))
    # plt.plot(preb['amount'])
    # plt.show()
    preb['amount'] = preb['amount']*1e3
    preb = get_frt_test(preb)
    preb['amount'] = preb['amount'] -500

    #填入测试结果
    test_day = pd.read_csv('data/test_v2_day.csv')
    for item in prea.index:
        test_day.loc[item * 2, 'amount'] = prea.loc[item, 'amount']
    for item in preb.index:
        if preb.loc[item, 'day'] in [5, 6]:
            test_day.loc[item * 2 + 1, 'amount'] = 0
        elif preb.loc[item, 'day'] in [0, 1, 2, 3, 4] and preb.loc[item, 'dayofyear'] > 361:
            test_day.loc[item * 2 + 1, 'amount'] = preb.loc[item, 'amount'] + 100
        else:
            test_day.loc[item * 2 + 1, 'amount'] = preb.loc[item, 'amount']

    # b = test_day[test_day.post_id == 'B'].copy()
    # a = test_day[test_day.post_id == 'A'].copy()
    # a = a.reset_index()
    # plt.plot(a.amount)
    # plt.show()
    # b = b.reset_index()
    # plt.plot(b.amount)
    # plt.show()
    test_day['amount'] = test_day['amount'].apply(lambda x: int(x))

    #保存结果
#     test_day.to_csv('submit/task1.txt', sep=',', encoding='utf-8', index=False)

                                     SARIMAX Results                                      
Dep. Variable:                                  y   No. Observations:                  281
Model:             SARIMAX(14, 1, 0)x(0, 1, 0, 7)   Log Likelihood                 259.259
Date:                            Wed, 12 May 2021   AIC                           -486.519
Time:                                    21:40:16   BIC                           -428.767
Sample:                                01-01-2018   HQIC                          -463.336
                                     - 10-08-2018                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0009      0.007      0.127      0.899      -0.012       0.014
ar.L1         -0.2583      0.016   

In [21]:
test_day

,date,post_id,amount
0,2020/12/1,A,18876
1,2020/12/1,B,3149
2,2020/12/2,A,16811
3,2020/12/2,B,3157
4,2020/12/3,A,16732
...,...,...,...
57,2020/12/29,B,3588
58,2020/12/30,A,21003
59,2020/12/30,B,3577
60,2020/12/31,A,21142
